# Flow-Shop Problem-Solving with Google `FunSearch`

> Authors: CUI Guangyuan, XU Zhuojun, LI Songyan

This notebook is the main entrance of our work on the flow-shop problems' solving using Google `FunSearch` via various attempts. Mainly, the applications of existing methods on the problem-solving can be divided into three categories: baseline experiments including the applications of Google `FunSearch` and `OR-Tools` as well as the existing heuristics (only NEH algorithm). New approaches are also proposed in this research, including:

- Trials on different kinds of prompts (Prompt Engineering)
- FunSearch with Curriculum Learning

## Dataset

## Baseline Experiments

### Existing Heuristics

In [ ]:
import time
import random
import math
import copy
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

# Parse the input data
def parse_input(input_data):
    lines = input_data.strip().split('\n')
    n_jobs, n_machines = map(int, lines[0].split())
    
    jobs = []
    for i in range(1, n_jobs + 1):
        job_data = lines[i].split()
        job = []
        for j in range(0, 2 * n_machines, 2):
            machine = int(job_data[j])
            processing_time = int(job_data[j + 1])
            job.append((machine, processing_time))
        jobs.append(job)
    
    return n_jobs, n_machines, jobs

# Calculate makespan for a given job sequence
def calculate_makespan(jobs, job_sequence, n_machines):
    machine_times = [0] * n_machines
    
    for job_idx in job_sequence:
        job = jobs[job_idx]
        for (machine, processing_time) in job:
            if machine == 0:
                machine_times[machine] = machine_times[machine] + processing_time
            else:
                machine_times[machine] = max(machine_times[machine], machine_times[machine - 1]) + processing_time
    
    return machine_times[-1]

# Function to visualize the schedule
def visualize_schedule(jobs, job_sequence, n_machines, algorithm_name):
    # Calculate start and end times for each operation
    machine_times = [0] * n_machines
    schedule = []
    
    for job_idx in job_sequence:
        job = jobs[job_idx]
        job_schedule = []
        
        for machine, processing_time in job:
            if machine == 0:
                start_time = machine_times[machine]
            else:
                start_time = max(machine_times[machine], machine_times[machine - 1])
            
            end_time = start_time + processing_time
            machine_times[machine] = end_time
            job_schedule.append((machine, start_time, end_time))
        
        schedule.append(job_schedule)
    
    # Create the visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Define colors for jobs
    colors = plt.cm.tab10.colors
    
    # Plot each operation
    for i, job_schedule in enumerate(schedule):
        job_idx = job_sequence[i]
        for machine, start, end in job_schedule:
            ax.barh(machine, end - start, left=start, height=0.8, 
                   color=colors[job_idx % len(colors)], alpha=0.8,
                   edgecolor='black', linewidth=1)
            
            # Add job number label
            if end - start > 30:  # Only add text if bar is wide enough
                ax.text(start + (end - start) / 2, machine, f'J{job_idx}', 
                       ha='center', va='center', color='black', fontweight='bold')
    
    # Add legend
    legend_elements = [Patch(facecolor=colors[i % len(colors)], edgecolor='black', label=f'Job {i}')
                      for i in range(len(jobs))]
    ax.legend(handles=legend_elements, loc='upper right')
    
    # Set labels and title
    ax.set_xlabel('Time')
    ax.set_ylabel('Machine')
    ax.set_yticks(range(n_machines))
    ax.set_yticklabels([f'Machine {i}' for i in range(n_machines)])
    ax.set_title(f'Flow Shop Schedule - {algorithm_name}\nMakespan: {machine_times[-1]}')
    
    # Add grid
    ax.grid(True, axis='x', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    return fig

# NEH Algorithm
def neh_algorithm(jobs, n_jobs, n_machines):
    # Calculate total processing time for each job
    job_times = []
    for i, job in enumerate(jobs):
        total_time = sum(time for _, time in job)
        job_times.append((i, total_time))
    
    # Sort jobs by total processing time (descending)
    job_times.sort(key=lambda x: x[1], reverse=True)
    
    # Build sequence incrementally
    sequence = [job_times[0][0]]
    
    for i in range(1, n_jobs):
        job_idx = job_times[i][0]
        best_makespan = float('inf')
        best_position = 0
        
        # Try inserting the job at each possible position
        for j in range(len(sequence) + 1):
            test_sequence = sequence.copy()
            test_sequence.insert(j, job_idx)
            makespan = calculate_makespan(jobs, test_sequence, n_machines)
            
            if makespan < best_makespan:
                best_makespan = makespan
                best_position = j
        
        sequence.insert(best_position, job_idx)
    
    makespan = calculate_makespan(jobs, sequence, n_machines)
    return sequence, makespan

# Main function to run and compare all algorithms
def main():
    # Test instance
    input_data = """11 5
0 375 1 12 2 142 3 245 4 412
0 632 1 452 2 758 3 278 4 398
0 12 1 876 2 124 3 534 4 765
0 460 1 542 2 523 3 120 4 499
0 528 1 101 2 789 3 124 4 999
0 796 1 245 2 632 3 375 4 123
0 532 1 230 2 543 3 896 4 452
0 14 1 124 2 214 3 543 4 785
0 257 1 527 2 753 3 210 4 463
0 896 1 896 2 214 3 258 4 259
0 532 1 302 2 501 3 765 4 988"""
    
    n_jobs, n_machines, jobs = parse_input(input_data)
    
    print(f"Problem: {n_jobs} jobs, {n_machines} machines")
    
    # Run and time each algorithm
    algorithms = [
        ("NEH Algorithm", lambda: neh_algorithm(jobs, n_jobs, n_machines)),
    ]
    
    results = []
    
    for name, algorithm in algorithms:
        print(f"\nRunning {name}...")
        start_time = time.time()
        sequence, makespan = algorithm()
        end_time = time.time()
        
        execution_time = end_time - start_time
        results.append((name, sequence, makespan, execution_time))
        
        print(f"  Sequence: {sequence}")
        print(f"  Makespan: {makespan}")
        print(f"  Time: {execution_time:.6f} seconds")
        
        # Create visualization
        fig = visualize_schedule(jobs, sequence, n_machines, name)
        plt.savefig(f"{name.replace(' ', '_')}_schedule.png")
        plt.show()
    
    # Compare results
    print("\nComparison of Algorithms:")
    print("-" * 60)
    print(f"{'Algorithm':<20} {'Makespan':<10} {'Time (s)':<15} {'Relative Quality':<15}")
    print("-" * 60)
    
    best_makespan = min(result[2] for result in results)
    
    for name, sequence, makespan, execution_time in results:
        relative_quality = (makespan / best_makespan - 1) * 100  # percentage above best
        print(f"{name:<20} {makespan:<10} {execution_time:<15.6f} {relative_quality:<15.2f}%")
    
    # Create comparison bar charts
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Makespan comparison
    algorithm_names = [result[0] for result in results]
    makespans = [result[2] for result in results]
    ax1.bar(algorithm_names, makespans, color='skyblue')
    ax1.set_title('Makespan Comparison')
    ax1.set_ylabel('Makespan')
    ax1.set_xticklabels(algorithm_names, rotation=45, ha='right')
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Execution time comparison (log scale)
    execution_times = [result[3] for result in results]
    ax2.bar(algorithm_names, execution_times, color='salmon')
    ax2.set_title('Execution Time Comparison')
    ax2.set_ylabel('Time (seconds)')
    ax2.set_yscale('log')
    ax2.set_xticklabels(algorithm_names, rotation=45, ha='right')
    ax2.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig("algorithm_comparison.png")
    plt.show()

if __name__ == "__main__":
    main()

### Google `OR-Tools`

Import necessary libraries:

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import numpy as np

from ortools.sat.python import cp_model

Define the function to read cases from instances in desired formats:

In [ ]:
def read_cases(path):
    cases = []
    with open(path, 'r') as f:
        alldata = f.readlines()
        first_line = alldata[0].split()
        n_jobs, n_machines = int(first_line[0]), int(first_line[1])

        for i in range(1, len(alldata)):
            line = alldata[i]
            jobs_cases = []
            data = line.split()
            for d in range(0, len(data), 2):
                jobs_cases.append((int(data[d]), int(data[d+1])))
            cases.append(jobs_cases)

    return (n_jobs, n_machines), cases

Define the function for plotting Gantt chart of the result:

In [ ]:
def plot_gantt_chart(result_job_schedule, num_jobs, num_machines,
                     title="Flow-Shop Gantt Chart"):
    fig, ax = plt.subplots(figsize=(25, 12))

    colors = list(mcolors.TABLEAU_COLORS.values())
    if num_jobs > len(colors):
        random.seed(4487)
        colors = []
        for _ in range(num_jobs):
            colors.append(f'#{random.randint(0, 0xFFFFFF):06x}')

    for job_id, machine_id, stime, etime in result_job_schedule:
        duration = etime - stime
        rect = patches.Rectangle(
            (stime, num_machines - machine_id - 1),
            duration,
            0.8,
            linewidth=1,
            edgecolor='black',
            facecolor=colors[job_id],
            alpha=0.6,
            label=f'Job-{job_id}' if machine_id == 0 else ''
        )
        ax.add_patch(rect)

        rx, ry = rect.get_xy()
        ax.text(
            rx + duration / 2,
            ry + 0.4,
            f'J{job_id}',
            ha='center',
            va='center',
            color='black',
            fontweight='light'
        )

    ax.set_xlim(0, max([endtime for _, _, _, endtime in result_job_schedule]) + 1)
    ax.set_ylim(0, num_machines)

    ax.set_yticks(np.arange(num_machines) + 0.4)
    ax.set_yticklabels([f'Machine {num_machines - i - 1}' for i in range(num_machines)])

    ax.grid(True, axis='x', linestyle='--', alpha=0.5)

    handles = [patches.Patch(color=colors[i], label=f'Job {i}') for i in range(num_jobs)]
    ax.legend(handles=handles, loc='upper right', ncol=min(5, num_jobs))

    ax.set_title(title)
    ax.set_xlabel('Time')
    ax.set_ylabel('Machines')

    plt.tight_layout()
    plt.show()

Define the main process of problem solving:

In [ ]:
def solve_flowshop(num_jobs, num_machines, jobs_data):
    model = cp_model.CpModel()


    """Create interval variables"""
    intervals = {}
    for job_id in range(num_jobs):
        for task_id, (machine_id, duration) in enumerate(jobs_data[job_id]):
            # a job is consisted of multiple tasks

            unique_id = f'{job_id}-{machine_id}-{task_id}'
            start = model.NewIntVar(0, 10000, f's-{unique_id}')
            end = model.NewIntVar(0, 10000, f'e-{unique_id}')
            interval = model.NewIntervalVar(start, duration, end, f'interval-{unique_id}')

            # uniquely identified by job ID and machine ID
            # since a task can only be executed on a machine
            intervals[(job_id, machine_id)] = (start, end, interval)


    """Add constraints on the order"""
    for job_id in range(num_jobs):
        for task_id in range(1, num_machines):
            # (task number = machine number) in flow-shop

            prev_task_machine = jobs_data[job_id][task_id - 1][0]
            cur_task_machine = jobs_data[job_id][task_id][0]

            # start time of current task must be larger than the end time of previous task
            model.Add(intervals[(job_id, cur_task_machine)][0]
                      >= intervals[(job_id, prev_task_machine)][1])


    """Add constraints on the machine conflicts"""
    for machine_id in range(num_machines):
        machine_intervals = [
            intervals[(job_id, machine_id)][2]
            for job_id in range(num_jobs)
        ]
        model.AddNoOverlap(machine_intervals)


    """Setting Objects"""
    case_object = model.NewIntVar(0, 10000, 'makespan')
    model.AddMaxEquality(case_object, [
        intervals[(job_id, num_machines-1)][1]
        for job_id in range(num_jobs)
    ])
    model.Minimize(case_object)


    """Solving"""
    solver = cp_model.CpSolver()
    stat = solver.Solve(model)


    """Results"""
    result_schedule = []
    if stat == cp_model.OPTIMAL:
        # exists optimal solution
        print(f'Optimal Makespan: {solver.ObjectiveValue()}')
        for job_id in range(num_jobs):
            for machine_id in range(num_machines):
                start = solver.Value(intervals[(job_id, machine_id)][0])
                duration = jobs_data[job_id][machine_id][1]
                end = start + duration
                print(f'Task-{machine_id} of Job-{job_id} is scheduled on Machine-{machine_id}: {start} ~ {end}')
                result_schedule.append((job_id, machine_id, start, end))
    else:
        # No optimal solution
        print('No solution found')

    return result_schedule


In [ ]:
case_set_name = 'reeves'
case_no = 20
case_path = f'../data/{case_set_name}/{case_set_name}{case_no}.txt'

(n_jobs, n_machines), jobs_data = read_cases(case_path)
result_schedule = solve_flowshop(n_jobs, n_machines, jobs_data)

plot_gantt_chart(result_schedule, n_jobs, n_machines, f'Result of case: {case_set_name}-{case_no}')

### Naïve Application on FunSearch

## New Approaches

### Prompt Engineering

### FunSearch with Curriculum Learning

This is our key improvements that has made on the basic FunSearch Framework. The main idea of the new framework is applying Curriculum Learning in the evolving process of FunSearch.

Specifically, a single iteration of evolving are now turned into multiple iterations. In this framework, we call them 'Stages':
- The input instances are divided into various stages, starting from 'easy' instances all the way up to 'complicated' instances
    - Degrees of complication are defined manually
- At each stage, FunSearch is executed only with the instances belong to that stage, and gets the result
- If the result of current stage is higher than the baseline score, then it enters the next stage, namely a more complicated stage
    - Baseline function that provides baseline score is given by the raw function at each stage before the actual evolving
- If the result of current stage is lower than the baseline score, it keeps trying until it reaches the maximum number of attempts defined in advance, or gets a better score and escape current stage
- The final output of this framework can either be the 'half-evolved' or 'completely-evolved' function due to the maximum attempts limit

> See directory `implementation_cl` for the detailed implementation of this framework.


## Evaluations